In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d danukhan/mnist-two-digit

 76% 40.0M/52.3M [00:00<00:00, 130MB/s]
100% 52.3M/52.3M [00:00<00:00, 127MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/mnist-two-digit.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow. keras import layers, regularizers
import os
import pandas as pd

In [5]:
BATCH_SIZE=64
WEIGHT_DECAY=0.001
LEARNING_RATE=0.001

In [6]:
train_df = pd.read_csv ("train.csv")
test_df = pd.read_csv("test.csv")
train_images = os.getcwd() + "/train_images/" + train_df. iloc[:, 0]. values
test_images = os.getcwd() + "/test_images/" + test_df. iloc[:, 0]. values
train_labels = train_df. iloc[:, 1:]. values
test_labels = test_df.iloc[:, 1:]. values

In [7]:
def read_image (image_path, label):
  image = tf.io.read_file (image_path)
  image = tf. image. decode_image (image, channels=1, dtype=tf.float32)
  # In older versions you need to set shape in order to avoid error
  # on newer (2.3.0+) the following 3 lines can safely be removed
  image. set_shape ((64, 64, 1))
  label[0].set_shape ([])
  label[1].set_shape ([])
  labels = {"first_num": label[0], "second_num": label[1]}
  return image, labels

In [8]:
AUTOTUNE = tf .data.experimental.AUTOTUNE

train_dataset = tf.data.Dataset.from_tensor_slices (
    (train_images, train_labels)
)

train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

In [9]:
test_dataset = tf .data. Dataset.from_tensor_slices ((test_images, test_labels))
test_dataset = (test_dataset
                .map (read_image)
                .batch (batch_size=BATCH_SIZE)
                .prefetch (buffer_size=AUTOTUNE)
)

In [10]:
inputs=layers.Input(shape=(64,64,1))
x=layers.Conv2D(filters=64,
                 kernel_size=3,
                 padding='same',
                 kernel_regularizer=regularizers.l2(WEIGHT_DECAY)
                 )(inputs)
x=layers.BatchNormalization()(x)
x=keras.activations.relu(x)

x=layers.Conv2D(filters=64,
                 kernel_size=3,
                 padding='same',
                 kernel_regularizer=regularizers.l2(WEIGHT_DECAY)
                 )(x)
x=layers.BatchNormalization()(x)
x=keras.activations.relu(x)
x=layers.MaxPool2D()(x)

x=layers.Conv2D(filters=64,
                 kernel_size=3,
                 padding='same',
                 kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
                 activation='relu'
                 )(x)

x=layers.Conv2D(filters=128,
                 kernel_size=3,
                 padding='same',
                 kernel_regularizer=regularizers.l2(WEIGHT_DECAY),
                 activation='relu'
                 )(x)
x=layers.MaxPool2D()(x)

x=layers.Flatten()(x)
x=layers.Dense(128,activation='relu')(x)
x=layers.Dropout(0.5)(x)
x=layers.Dense(64,activation='relu')(x)

output1=layers.Dense(10,name='first_num')(x) #if we are using labels as list of dict then name here should be same
                                              # as dict in labels name
output2=layers.Dense(10,name='second_num')(x)

model=keras.Model(inputs=inputs, outputs=[output1,output2])

In [11]:
model.compile(optimizer=keras.optimizers.Adam(LEARNING_RATE),
              loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=True), keras.losses.SparseCategoricalCrossentropy(from_logits=True)],
              metrics=['accuracy'])

In [12]:
model.fit(train_dataset,
          epochs=10,
          verbose=2)

Epoch 1/10
1000/1000 - 47s - loss: 1.8482 - first_num_loss: 0.8623 - second_num_loss: 0.8367 - first_num_accuracy: 0.6988 - second_num_accuracy: 0.7062 - 47s/epoch - 47ms/step
Epoch 2/10
1000/1000 - 37s - loss: 0.5201 - first_num_loss: 0.2113 - second_num_loss: 0.2071 - first_num_accuracy: 0.9328 - second_num_accuracy: 0.9336 - 37s/epoch - 37ms/step
Epoch 3/10
1000/1000 - 36s - loss: 0.3553 - first_num_loss: 0.1381 - second_num_loss: 0.1302 - first_num_accuracy: 0.9561 - second_num_accuracy: 0.9598 - 36s/epoch - 36ms/step
Epoch 4/10
1000/1000 - 37s - loss: 0.2815 - first_num_loss: 0.1059 - second_num_loss: 0.0965 - first_num_accuracy: 0.9668 - second_num_accuracy: 0.9692 - 37s/epoch - 37ms/step
Epoch 5/10
1000/1000 - 36s - loss: 0.2462 - first_num_loss: 0.0888 - second_num_loss: 0.0818 - first_num_accuracy: 0.9719 - second_num_accuracy: 0.9735 - 36s/epoch - 36ms/step
Epoch 6/10
1000/1000 - 36s - loss: 0.2236 - first_num_loss: 0.0767 - second_num_loss: 0.0733 - first_num_accuracy: 0.975

In [13]:
model.evaluate(test_dataset,
               verbose=2)

313/313 - 4s - loss: 0.7558 - first_num_loss: 0.2862 - second_num_loss: 0.4077 - first_num_accuracy: 0.9265 - second_num_accuracy: 0.8971 - 4s/epoch - 12ms/step


[0.7558404207229614,
 0.2862430810928345,
 0.4077312648296356,
 0.9264500141143799,
 0.8970999717712402]